In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/cassandra/cascade-cup-22.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [27]:
df1=pd.read_csv('/tmp/train.csv')

In [74]:
df = df1.copy()
df.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [75]:
(df.isnull().sum()/len(df))*100

order_time               0.000000
order_id                 0.000000
order_date               0.000000
allot_time               0.000000
accept_time              0.034889
pickup_time              0.538000
delivered_time           1.159556
rider_id                 0.000000
first_mile_distance      0.000000
last_mile_distance       0.000000
alloted_orders           3.766222
delivered_orders         3.853556
cancelled                0.000000
undelivered_orders       3.853556
lifetime_order_count     0.011778
reassignment_method     96.945778
reassignment_reason     96.943778
reassigned_order        96.943778
session_time             0.816667
cancelled_time          98.840444
dtype: float64

In [76]:
df = df[['order_time','accept_time','rider_id','first_mile_distance','last_mile_distance','delivered_orders','undelivered_orders','cancelled','lifetime_order_count','session_time','reassigned_order']]
df.head()

,order_time,accept_time,rider_id,first_mile_distance,last_mile_distance,delivered_orders,undelivered_orders,cancelled,lifetime_order_count,session_time,reassigned_order
0,2021-01-26 02:21:35,2021-01-26 02:22:08,11696,1.5666,2.65,46.0,0.0,0,621.0,NaN,NaN
1,2021-01-26 02:33:16,2021-01-26 02:34:45,18117,2.5207,2.76,8.0,0.0,0,105.0,3.266667,NaN
2,2021-01-26 02:39:49,2021-01-26 02:40:13,18623,2.2074,4.80,1.0,0.0,0,66.0,9.816667,NaN
3,2021-01-26 02:47:53,2021-01-26 02:49:06,15945,2.1894,6.38,1.0,0.0,0,127.0,17.533333,NaN
4,2021-01-26 03:06:30,2021-01-26 03:07:57,17589,2.7870,4.01,34.0,0.0,0,84.0,1.350000,NaN


In [77]:
delivered = int(df.delivered_orders.mean())
lifetime_order = int(df.lifetime_order_count.mean())
undelivered_orders = int(df.undelivered_orders.mean())
session_time = int(df.session_time.mean())

df['delivered_orders'] = df['delivered_orders'].fillna(delivered)
df['lifetime_order_count'] = df['lifetime_order_count'].fillna(lifetime_order)
df['undelivered_orders'] = df['undelivered_orders'].fillna(undelivered_orders)
df['session_time'] = df['session_time'].fillna(session_time)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [78]:
# extract Hours from Timestamp
df['order_time'] = df['order_time'].astype('datetime64[ns]')
df['Hour'] = df.order_time.dt.hour

# Day of week
df['weekday']=df['order_time'].dt.day_name()
dictionary={'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}
df['weekday']=df['weekday'].map(dictionary)

# value is 1 when NAN else 0
df['accept_time_null']=np.where(df['accept_time'].isnull(),1,0)
# fill 0 inplace of nan
df['reassigned_order'] = df['reassigned_order'].fillna(0)

# drop useless column
x_train = df.drop(['accept_time' , 'cancelled','order_time','rider_id','reassigned_order'] , axis=1)
y_train = df['cancelled']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [79]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: cancelled, dtype: int64

In [80]:
x_train.head()

,first_mile_distance,last_mile_distance,delivered_orders,undelivered_orders,lifetime_order_count,session_time,Hour,weekday,accept_time_null
0,1.5666,2.65,46.0,0.0,621.0,220.000000,2,2,0
1,2.5207,2.76,8.0,0.0,105.0,3.266667,2,2,0
2,2.2074,4.80,1.0,0.0,66.0,9.816667,2,2,0
3,2.1894,6.38,1.0,0.0,127.0,17.533333,2,2,0
4,2.7870,4.01,34.0,0.0,84.0,1.350000,3,2,0


In [81]:
# Normalizing data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
standardized_data = scaler.transform(x_train)
print(standardized_data)
x_train = standardized_data

[[ 0.39791812 -0.1692421  -0.65889323 ... -2.96268743 -1.0822076
  -0.01868183]
 [ 1.52545378 -0.11085946 -1.09122426 ... -2.96268743 -1.0822076
  -0.01868183]
 [ 1.15520231  0.97187317 -1.17086419 ... -2.96268743 -1.0822076
  -0.01868183]
 ...
 [ 0.43077159 -0.72122344 -0.86368161 ... -0.75251012  1.2139591
  -0.01868183]
 [ 1.96649382  0.90818301 -0.36308778 ... -0.75251012  1.2139591
  -0.01868183]
 [ 0.78306001 -1.5279654  -0.84092735 ... -0.75251012  1.2139591
  -0.01868183]]


In [82]:
y_train.value_counts()

0    444782
1      5218
Name: cancelled, dtype: int64

In [83]:
# removing imbalance of data
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(x_train, y_train)

y_sm.value_counts()

0    444782
1    444782
Name: cancelled, dtype: int64

In [85]:
# training model

import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(9, input_shape=(9,), activation='relu'),
    keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_sm, y_sm, epochs=5)

Epoch 1/5
27799/27799 [==============================] - 59s 2ms/step - loss: 0.6357 - accuracy: 0.6234
Epoch 2/5
27799/27799 [==============================] - 58s 2ms/step - loss: 0.6243 - accuracy: 0.6364
Epoch 3/5
27799/27799 [==============================] - 60s 2ms/step - loss: 0.6226 - accuracy: 0.6352
Epoch 4/5
27799/27799 [==============================] - 58s 2ms/step - loss: 0.6213 - accuracy: 0.6351
Epoch 5/5
27799/27799 [==============================] - 59s 2ms/step - loss: 0.6206 - accuracy: 0.6355


In [86]:
test=pd.read_csv('/tmp/test.csv')
x_test = test.copy()

x_test = x_test[['order_time','accept_time','rider_id','first_mile_distance','last_mile_distance','delivered_orders','undelivered_orders','session_time' , 'lifetime_order_count','reassigned_order']]

# alloted = int(x_test.alloted_orders.mean())
delivered = int(x_test.delivered_orders.mean())
lifetime_order = int(x_test.lifetime_order_count.mean())
undelivered_orders = int(x_test.undelivered_orders.mean())
session_time = int(x_test.session_time.mean())

# x_test['alloted_orders'] = x_test['alloted_orders'].fillna(alloted)
x_test['delivered_orders'] = x_test['delivered_orders'].fillna(delivered)
x_test['lifetime_order_count'] = x_test['lifetime_order_count'].fillna(lifetime_order)
x_test['undelivered_orders'] = x_test['undelivered_orders'].fillna(undelivered_orders)
x_test['session_time'] = x_test['session_time'].fillna(session_time)

# extract Hours from Timestamp
x_test['order_time'] = x_test['order_time'].astype('datetime64[ns]')
x_test['Hour'] = x_test.order_time.dt.hour

# Day of week
x_test['weekday']=x_test['order_time'].dt.day_name()
dictionary={'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}
x_test['weekday']=x_test['weekday'].map(dictionary)

# where accept_time is null value = 1 else value = 0
x_test['accept_time_null']=np.where(x_test['accept_time'].isnull(),1,0)

# drop useless column
x_test = x_test.drop(['order_time' , 'accept_time' , 'accept_time' ,'rider_id','reassigned_order'] , axis=1)


In [87]:
x_test.head()

,first_mile_distance,last_mile_distance,delivered_orders,undelivered_orders,session_time,lifetime_order_count,Hour,weekday,accept_time_null
0,1.6585,4.54,215.0,1.0,273.400000,747.0,10,6,0
1,2.0709,5.84,52.0,0.0,252.100000,75.0,10,6,0
2,1.3884,0.99,289.0,0.0,241.383333,2214.0,10,6,0
3,1.9039,2.59,122.0,3.0,291.933333,1020.0,10,6,0
4,0.8275,0.94,350.0,2.0,247.133333,7284.0,10,6,0


In [88]:
# removing imbalance of test data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_test)
standardized_ = scaler.transform(x_test)
print(standardized_)
x_test = standardized_

[[ 0.39934402  0.92452277  1.2580881  ... -0.93181998  0.8676972
  -0.02167236]
 [ 0.88299593  1.63410855 -0.59781684 ... -0.93181998  0.8676972
  -0.02167236]
 [ 0.08257782 -1.01319224  2.10064617 ... -0.93181998  0.8676972
  -0.02167236]
 ...
 [ 0.85778129 -0.86581673 -0.31316884 ...  2.02343434 -0.68808957
  -0.02167236]
 [-1.45130005  1.44852458  1.40610506 ...  2.02343434 -0.68808957
  -0.02167236]
 [ 0.45047696  0.70073033 -0.87107892 ...  2.02343434 -0.68808957
  -0.02167236]]


In [89]:
# Predicting the Test set results  
yp = model.predict(x_test)

y_pred = []
for element in yp:
    if element >= 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [90]:
(unique, counts) = np.unique(y_pred, return_counts=True)
frequencies = np.asarray((unique, counts)).T

In [91]:
frequencies

array([[     0, 110585],
       [     1,  34259]])

In [92]:
order_id = list(test['order_id'])

In [93]:
dict = {'order_id': order_id, 'cancelled': y_pred}  
       
result = pd.DataFrame(dict) 
    
# saving the dataframe 
result.to_csv('file.csv', index=False)

In [94]:
# from google.colab import files
# files.download("file.csv")